<a href="https://colab.research.google.com/github/bh8648/Deep-Learning/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/Deep-Learning/

/content/drive/MyDrive/Deep-Learning


In [9]:
# <학습자료 8-1>

# define_argparser 함수
import argparse

import torch
import torch.nn as nn
import torch.optim as optim

from model import ImageClassifier
from trainer import Trainer

from utils import load_mnist
from utils import split_data
from utils import get_hidden_sizes


def define_argparser() :
    p = argparse.ArgumentParser()

    p.add_argument('--model_fn', required=True)
    p.add_argument('--gpu_id', type=int, default=0 if torch.cuda.is_available() else -1)

    p.add_argument('--train_ratio', type=float, default=.8)

    p.add_argument('--batch_size', type=int, default=256)
    p.add_argument('--n_epochs', type=int, default=20)

    p.add_argument('--n_layers', type=int, default=5)
    p.add_argument('--use_dropout', action='store_true')
    p.add_argument('--dropout_p', type=float, default=5)

    # verbose => 여러가지 내용들을 장황하게 다 보여주는 것
    p. add_argument('--verbose', type=int, default=1)

    config = p.parse_args()

    return config


def main(config) :
    # Set device based on user defined configuration.
    device = torch.device('cpu') if config.gpu_id < 0 else torch.device('cuda:%d' % config.gpu_id)
    
    x, y = load_mnist(is_train=True, flatten=True)
    x, y = split_data(x.to(device), y.to(device), train_ratio=config.train_ratio)
    
    print("Train:", x[0].shape, y[0].shape)
    print("Valid:", x[1].shape, y[1].shape)
    
    input_size = int(x[0].shape[-1])
    output_size = int(max(y[0])) + 1

    model = ImageClassifier(
        input_size = input_size,
        output_size = output_size,
        hidden_sizes = get_hidden_sizes(input_size,
                                       output_size,
                                       config.n_layers),
        use_batch_norm = not config.use_dropout,
        dropout_p = config.dropout_p,
    ).to(device)
    optimizer = optim.Adam(model.parameters())
    crit = nn.NLLLoss
    
    if config.verbose >= 1 :
        print(model)
        print(optimizer)
        print(crit)

    trainer = Trainer(model, optimizer, crit)
    
    # 트레이닝이 일어남.
    trainer.train(
        train_data(x[0], y[0]),
        train_data(x[0], y[0]),
        config = config
    )
    
    # Save best model weights.
    torch.save({
        'model' : trainer.model.state_dict(),
        'opt' : optimizer.state_dict(),
        'config' : config
    }, config.model_fn)

if __name__ == '__main__' :
    config = define_argparser()
    main(config)

usage: ipykernel_launcher.py [-h] --model_fn MODEL_FN [--gpu_id GPU_ID]
                             [--train_ratio TRAIN_RATIO]
                             [--batch_size BATCH_SIZE] [--n_epochs N_EPOCHS]
                             [--n_layers N_LAYERS] [--use_dropout]
                             [--dropout_p DROPOUT_P] [--verbose VERBOSE]
ipykernel_launcher.py: error: the following arguments are required: --model_fn


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
